In [1]:
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Tuple, Dict
import unittest

from tensorflow.keras.layers import Embedding, MultiHeadAttention, Dense, Input, Dropout, LayerNormalization

In [2]:
import numpy as np

def get_angles(pos: np.ndarray, k: np.ndarray, d: int) -> np.ndarray:
    """
    Compute the raw angle terms for positional encoding.

    Formula (from "Attention is All You Need"):
        angle(pos, k) = pos / (10000^(2i/d))

    where:
        pos (np.ndarray): Column vector of positions, shape (N, 1)
                          e.g., [[0], [1], ..., [N-1]]
        k   (np.ndarray): Row vector of dimension indices, shape (1, d)
                          e.g., [[0, 1, ..., d-1]]
        d   (int): Dimension of the embedding/positional encoding.
        i   (np.ndarray): Frequency index, computed as k // 2.

    Returns:
        np.ndarray: Angle matrix of shape (N, d), where each row
                    corresponds to a position and each column to a dimension.
    """
    i = k // 2
    angles = pos / np.power(10000, (2 * i) / float(d))
    return angles


In [3]:
class TestGetAngles(unittest.TestCase):

    def test_pos_zero(self):
        pos = np.array([[0]])
        k = np.arange(8)[np.newaxis, :]
        angles = get_angles(pos, k, 8)
        self.assertTrue(np.allclose(angles, np.zeros((1, 8))))

    def test_small_values(self):
        pos = np.array([[2]])
        k = np.arange(4)[np.newaxis, :]
        angles = get_angles(pos, k, 4)
        expected = np.array([[2.0, 2.0, 0.02, 0.02]])
        self.assertTrue(np.allclose(angles, expected, atol=1e-4))

    def test_symmetry_even_odd(self):
        pos = np.array([[5]])
        k = np.arange(6)[np.newaxis, :]
        angles = get_angles(pos, k, 6)
        self.assertTrue(np.allclose(angles[0,4], angles[0,5]))

    def test_increasing_d(self):
        pos = np.array([[10]])
        k = np.array([[2]])
        small_d = get_angles(pos, k, 8)
        large_d = get_angles(pos, k, 512)
        self.assertGreater(large_d, small_d)

unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestGetAngles))

....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

In [4]:
def positional_encoding(positions: int, d: int) -> tf.Tensor:
    """
    Precomputes a matrix with all the positional encodings.

    Arguments:
        positions (int) -- Maximum number of positions to be encoded
        d (int) -- Encoding size

    Returns:
        pos_encoding (tf.Tensor) -- Tensor of shape (1, positions, d) containing the positional encodings
    """
    # Create position indices (positions, 1)
    pos: np.ndarray = np.arange(positions)[:, np.newaxis]

    # Create dimension indices (1, d)
    k: np.ndarray = np.arange(d)[np.newaxis, :]

    # Compute raw angles using get_angles
    angle_rads: np.ndarray = get_angles(pos, k, d)

    # Apply sin to even indices (2i)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # Apply cos to odd indices (2i+1)
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    # Add batch dimension
    pos_encoding: np.ndarray = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


In [5]:
class TestPositionalEncoding(unittest.TestCase):

    def test_shape(self):
        positions = 10
        d = 8
        pos_enc = positional_encoding(positions, d)
        # Check output shape (1, positions, d)
        self.assertEqual(pos_enc.shape, (1, positions, d))

    def test_sin_cos_ranges(self):
        positions = 5
        d = 6
        pos_enc = positional_encoding(positions, d).numpy()  # convert to NumPy for easy checks

        # Even indices: sin
        even_vals = pos_enc[0, :, 0::2]
        # Should be between -1 and 1
        self.assertTrue(np.all(even_vals >= -1.0) and np.all(even_vals <= 1.0))

        # Odd indices: cos
        odd_vals = pos_enc[0, :, 1::2]
        self.assertTrue(np.all(odd_vals >= -1.0) and np.all(odd_vals <= 1.0))

    def test_batch_dimension(self):
        positions = 7
        d = 4
        pos_enc = positional_encoding(positions, d)
        # Batch size should be 1
        self.assertEqual(pos_enc.shape[0], 1)

# Run the tests in Colab
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestPositionalEncoding))


...
----------------------------------------------------------------------
Ran 3 tests in 0.033s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [6]:
def create_padding_mask(decoder_token_ids: tf.Tensor) -> tf.Tensor:
    """
    Creates a padding mask for Transformer attention.

    This mask will have 1s for real tokens and 0s for padding tokens (usually 0 in the input),
    and adds an extra dimension to allow broadcasting in the attention mechanism.

    Arguments:
        decoder_token_ids (tf.Tensor): Tensor of shape (batch_size, seq_len) containing token IDs,
                                       where padding tokens are 0.

    Returns:
        tf.Tensor: Binary mask of shape (batch_size, 1, seq_len), where
                   1 indicates a real token (attend to it) and
                   0 indicates padding (ignore in attention).
    """
    # 1 for real tokens, 0 for padding tokens
    seq_mask = 1 - tf.cast(tf.math.equal(decoder_token_ids, 0), tf.float32)

    # Add extra dimension for broadcasting in attention
    return seq_mask[:, tf.newaxis, :]


def create_look_ahead_mask(sequence_length: int) -> tf.Tensor:
    """
    Creates a look-ahead mask for decoder self-attention.

    The mask prevents a position from attending to future positions.
    Returns a lower-triangular matrix of ones with shape (1, sequence_length, sequence_length).

    Arguments:
        sequence_length (int): Length of the sequence to mask.

    Returns:
        tf.Tensor: Tensor of shape (1, sequence_length, sequence_length) where
                   1 indicates allowed attention and 0 indicates masked future positions.
    """
    mask = tf.linalg.band_part(tf.ones((1, sequence_length, sequence_length)), -1, 0)
    return mask

In [7]:
def scaled_dot_product_attention(q: tf.Tensor, k: tf.Tensor, v: tf.Tensor, mask: tf.Tensor = None):
    """
    Calculate the scaled dot-product attention.

    Arguments:
        q -- query tensor of shape (..., seq_len_q, depth)
        k -- key tensor of shape (..., seq_len_k, depth)
        v -- value tensor of shape (..., seq_len_v, depth_v)
        mask -- Float tensor broadcastable to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
        output -- attention output tensor of shape (..., seq_len_q, depth_v)
        attention_weights -- attention weights tensor of shape (..., seq_len_q, seq_len_k)
    """
    # Dot product of queries and keys
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    # Scale by sqrt(depth of key)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # Add mask (if any)
    if mask is not None:
        # mask contains 1s for valid positions, 0s for masked positions
        # multiply by -1e9 so that softmax of masked positions ≈ 0
        scaled_attention_logits += (mask - 1) * 1e9

    # Softmax along last axis (seq_len_k)
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    # Multiply by values
    output = tf.matmul(attention_weights, v)

    return output, attention_weights


In [8]:
class TestScaledDotProductAttention(unittest.TestCase):

    def test_output_shapes(self):
        # q, k, v shapes: (batch_size, seq_len, depth)
        q = tf.constant(np.random.rand(2, 3, 4), dtype=tf.float32)
        k = tf.constant(np.random.rand(2, 3, 4), dtype=tf.float32)
        v = tf.constant(np.random.rand(2, 3, 5), dtype=tf.float32)  # depth_v = 5

        output, attn_weights = scaled_dot_product_attention(q, k, v)

        # Check output shapes
        self.assertEqual(output.shape, (2, 3, 5))
        self.assertEqual(attn_weights.shape, (2, 3, 3))

    def test_attention_sum_to_one(self):
        # Attention weights along last axis should sum to 1
        q = tf.constant([[ [1., 0.], [0., 1.] ]], dtype=tf.float32)  # shape (1, 2, 2)
        k = tf.constant([[ [1., 0.], [0., 1.] ]], dtype=tf.float32)
        v = tf.constant([[ [1., 2.], [3., 4.] ]], dtype=tf.float32)

        _, attn_weights = scaled_dot_product_attention(q, k, v)

        # Sum along last axis should be 1
        attn_sum = tf.reduce_sum(attn_weights, axis=-1)
        self.assertTrue(np.allclose(attn_sum.numpy(), np.ones(attn_sum.shape), atol=1e-6))

    def test_masking_effect(self):
        # Mask should zero out future tokens
        q = tf.constant([[ [1., 0.], [0., 1.], [1., 1.] ]], dtype=tf.float32)
        k = q
        v = tf.constant([[ [1., 0.], [0., 1.], [1., 1.] ]], dtype=tf.float32)

        # Create a look-ahead mask
        mask = tf.constant([[ [1., 0., 0.],
                              [1., 1., 0.],
                              [1., 1., 1.] ]], dtype=tf.float32)

        output, attn_weights = scaled_dot_product_attention(q, k, v, mask)

        # Masked positions in attention weights should be ~0
        self.assertTrue(attn_weights[0,0,1] < 1e-6)
        self.assertTrue(attn_weights[0,0,2] < 1e-6)
        self.assertTrue(attn_weights[0,1,2] < 1e-6)

# Run the tests
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestScaledDotProductAttention))


...
----------------------------------------------------------------------
Ran 3 tests in 0.104s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [9]:
class FullyConnected(tf.keras.layers.Layer):
    """
    Simple position-wise feed-forward network for Transformer layers.

    Architecture:
        Dense(fully_connected_dim, ReLU) -> Dense(embedding_dim)
    """
    def __init__(self, embedding_dim: int, fully_connected_dim: int):
        """
        Arguments:
            embedding_dim (int): Dimension of the Transformer embedding (input & output).
            fully_connected_dim (int): Hidden layer dimension for the feed-forward network.
        """
        super().__init__()
        self.dense1 = Dense(fully_connected_dim, activation='relu')
        self.dense2 = Dense(embedding_dim)

    def call(self, x: tf.Tensor) -> tf.Tensor:
        """
        Forward pass through the feed-forward network.

        Arguments:
            x (tf.Tensor): Input tensor of shape (batch_size, seq_len, embedding_dim)

        Returns:
            tf.Tensor: Output tensor of shape (batch_size, seq_len, embedding_dim)
        """
        x = self.dense1(x)
        x = self.dense2(x)
        return x

class EncoderLayer(tf.keras.layers.Layer):
    """
    Single Transformer Encoder layer consisting of:
    - Multi-head self-attention
    - Feed-forward network
    - Residual connections + Layer Normalization
    """
    def __init__(self,
                 embedding_dim: int,
                 num_heads: int,
                 fully_connected_dim: int,
                 dropout_rate: float = 0.1,
                 layernorm_eps: float = 1e-6):
        """
        Arguments:
            embedding_dim (int): Dimension of the input embeddings.
            num_heads (int): Number of attention heads in multi-head attention.
            fully_connected_dim (int): Hidden layer size for the feed-forward network.
            dropout_rate (float, optional): Dropout rate for feed-forward output. Defaults to 0.1.
            layernorm_eps (float, optional): Epsilon for LayerNormalization. Defaults to 1e-6.
        """
        super().__init__()

        self.mha = MultiHeadAttention(num_heads=num_heads,
                                      key_dim=embedding_dim,
                                      dropout=dropout_rate)

        self.ffn = FullyConnected(embedding_dim=embedding_dim,
                                  fully_connected_dim=fully_connected_dim)

        self.layernorm1 = LayerNormalization(epsilon=layernorm_eps)
        self.layernorm2 = LayerNormalization(epsilon=layernorm_eps)

        self.dropout_ffn = Dropout(dropout_rate)

    def call(self, x: tf.Tensor, training: bool, mask: Optional[tf.Tensor] = None) -> tf.Tensor:
        """
        Forward pass for the Encoder Layer.

        Arguments:
            x (tf.Tensor): Input tensor of shape (batch_size, seq_len, embedding_dim)
            training (bool): Whether the model is in training mode (applies dropout if True)
            mask (tf.Tensor, optional): Attention mask of shape broadcastable to
                                        (batch_size, num_heads, seq_len, seq_len). Defaults to None.

        Returns:
            tf.Tensor: Output tensor of shape (batch_size, seq_len, embedding_dim)
        """
        # Self-attention
        self_mha_output = self.mha(query=x, value=x, key=x, attention_mask=mask)

        # Skip connection + LayerNorm
        skip_x_attention = self.layernorm1(x + self_mha_output)

        # Feed-forward network
        ffn_output = self.ffn(skip_x_attention)

        # Dropout
        ffn_output = self.dropout_ffn(ffn_output, training=training)

        # Skip connection + LayerNorm
        encoder_layer_out = self.layernorm2(skip_x_attention + ffn_output)

        return encoder_layer_out


In [10]:
class TestEncoderLayer(unittest.TestCase):
    def test_encoder_layer_output_shape(self):
        batch_size = 2
        seq_len = 4
        embedding_dim = 8
        num_heads = 2
        ffn_dim = 16

        encoder_layer = EncoderLayer(embedding_dim, num_heads, ffn_dim)

        x = tf.constant(np.random.rand(batch_size, seq_len, embedding_dim), dtype=tf.float32)
        mask = tf.constant(np.ones((batch_size, 1, 1, seq_len)), dtype=tf.float32)

        output = encoder_layer(x, training=False, mask=mask)

        # Check output shape
        self.assertEqual(output.shape, (batch_size, seq_len, embedding_dim))

    def test_encoder_layer_no_mask(self):
        batch_size = 1
        seq_len = 3
        embedding_dim = 4
        num_heads = 1
        ffn_dim = 8

        encoder_layer = EncoderLayer(embedding_dim, num_heads, ffn_dim)
        x = tf.constant(np.random.rand(batch_size, seq_len, embedding_dim), dtype=tf.float32)

        output = encoder_layer(x, training=False)
        self.assertEqual(output.shape, (batch_size, seq_len, embedding_dim))

# Run tests
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestEncoderLayer))


..
----------------------------------------------------------------------
Ran 2 tests in 0.379s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [11]:
class Encoder(tf.keras.layers.Layer):
    """
    Full Transformer Encoder consisting of:
    1. Token embedding
    2. Positional encoding
    3. A stack of EncoderLayers
    4. Dropout for regularization

    The encoder transforms token indices into embeddings, adds positional information,
    and passes the result through multiple encoder layers for contextual representation.
    """
    def __init__(self,
                 num_layers: int,
                 embedding_dim: int,
                 num_heads: int,
                 fully_connected_dim: int,
                 input_vocab_size: int,
                 maximum_position_encoding: int,
                 dropout_rate: float = 0.1,
                 layernorm_eps: float = 1e-6):
        """
        Initializes the Transformer Encoder.

        Arguments:
            num_layers (int): Number of stacked EncoderLayer blocks.
            embedding_dim (int): Dimensionality of the token embeddings.
            num_heads (int): Number of attention heads for multi-head attention.
            fully_connected_dim (int): Hidden size of the feed-forward network in each EncoderLayer.
            input_vocab_size (int): Size of the input vocabulary (number of unique tokens).
            maximum_position_encoding (int): Maximum number of positions to encode.
            dropout_rate (float, optional): Dropout rate applied to embeddings and FFN. Defaults to 0.1.
            layernorm_eps (float, optional): Epsilon value for LayerNormalization. Defaults to 1e-6.
        """
        super().__init__()

        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        # Token embedding layer
        self.embedding = Embedding(input_vocab_size, self.embedding_dim)

        # Positional encoding matrix
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.embedding_dim)

        # Stack of EncoderLayer blocks
        self.enc_layers = [
            EncoderLayer(
                embedding_dim=self.embedding_dim,
                num_heads=num_heads,
                fully_connected_dim=fully_connected_dim,
                dropout_rate=dropout_rate,
                layernorm_eps=layernorm_eps
            )
            for _ in range(num_layers)
        ]

        # Dropout layer applied after embedding
        self.dropout = Dropout(dropout_rate)

    def call(self, x: tf.Tensor, training: bool, mask: Optional[tf.Tensor] = None) -> tf.Tensor:
        """
        Forward pass through the Transformer Encoder.

        Arguments:
            x (tf.Tensor): Input tensor of shape (batch_size, seq_len), containing token indices.
            training (bool): Boolean flag indicating training mode (for dropout).
            mask (tf.Tensor, optional): Attention mask of shape broadcastable to
                                        (batch_size, num_heads, seq_len, seq_len). Defaults to None.

        Returns:
            tf.Tensor: Output tensor of shape (batch_size, seq_len, embedding_dim),
                       representing contextualized embeddings for each token.
        """
        seq_len = tf.shape(x)[1]

        # Token embedding
        x = self.embedding(x)  # (batch_size, seq_len, embedding_dim)

        # Scale embeddings
        x *= tf.math.sqrt(tf.cast(self.embedding_dim, tf.float32))

        # Add positional encoding
        x += self.pos_encoding[:, :seq_len, :]

        # Apply dropout
        x = self.dropout(x, training=training)

        # Pass through stacked EncoderLayers
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training=training, mask=mask)

        return x  # (batch_size, seq_len, embedding_dim)


In [12]:
class TestEncoder(unittest.TestCase):
    def test_encoder_output_shape(self):
        batch_size = 2
        seq_len = 5
        vocab_size = 50
        embedding_dim = 16
        num_layers = 2
        num_heads = 4
        ffn_dim = 32
        max_pos_encoding = 10

        encoder = Encoder(
            num_layers=num_layers,
            embedding_dim=embedding_dim,
            num_heads=num_heads,
            fully_connected_dim=ffn_dim,
            input_vocab_size=vocab_size,
            maximum_position_encoding=max_pos_encoding
        )

        # Random token IDs
        x_input = tf.constant(np.random.randint(0, vocab_size, (batch_size, seq_len)), dtype=tf.int32)

        # Forward pass
        output = encoder(x_input, training=False, mask=None)

        # Check output shape
        self.assertEqual(output.shape, (batch_size, seq_len, embedding_dim))

unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestEncoder))


.
----------------------------------------------------------------------
Ran 1 test in 0.729s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [13]:
class DecoderLayer(tf.keras.layers.Layer):
    """
    Transformer Decoder Layer consisting of:
    1. Masked self-attention (with look-ahead mask)
    2. Encoder-Decoder attention (using encoder output)
    3. Feed-forward network
    Each sub-layer uses residual connections followed by layer normalization.
    """
    def __init__(self,
                 embedding_dim: int,
                 num_heads: int,
                 fully_connected_dim: int,
                 dropout_rate: float = 0.1,
                 layernorm_eps: float = 1e-6):
        """
        Initializes the DecoderLayer.

        Arguments:
            embedding_dim (int): Dimensionality of the embeddings.
            num_heads (int): Number of attention heads.
            fully_connected_dim (int): Hidden size of the feed-forward network.
            dropout_rate (float, optional): Dropout rate applied to FFN. Defaults to 0.1.
            layernorm_eps (float, optional): Epsilon value for LayerNormalization. Defaults to 1e-6.
        """
        super().__init__()

        self.mha1 = MultiHeadAttention(num_heads=num_heads,
                                       key_dim=embedding_dim,
                                       dropout=dropout_rate)
        self.mha2 = MultiHeadAttention(num_heads=num_heads,
                                       key_dim=embedding_dim,
                                       dropout=dropout_rate)

        self.ffn = FullyConnected(embedding_dim=embedding_dim,
                                  fully_connected_dim=fully_connected_dim)

        self.layernorm1 = LayerNormalization(epsilon=layernorm_eps)
        self.layernorm2 = LayerNormalization(epsilon=layernorm_eps)
        self.layernorm3 = LayerNormalization(epsilon=layernorm_eps)

        self.dropout_ffn = Dropout(dropout_rate)

    def call(self,
             x: tf.Tensor,
             enc_output: tf.Tensor,
             training: bool,
             look_ahead_mask: Optional[tf.Tensor] = None,
             padding_mask: Optional[tf.Tensor] = None
            ) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
        """
        Forward pass for the Decoder Layer.

        Arguments:
            x (tf.Tensor): Input tensor of shape (batch_size, target_seq_len, embedding_dim).
            enc_output (tf.Tensor): Encoder output of shape (batch_size, input_seq_len, embedding_dim).
            training (bool): Flag to enable dropout during training.
            look_ahead_mask (tf.Tensor, optional): Mask to prevent attention to future positions.
            padding_mask (tf.Tensor, optional): Mask to ignore padding in encoder-decoder attention.

        Returns:
            Tuple containing:
                out3 (tf.Tensor): Output tensor (batch_size, target_seq_len, embedding_dim).
                attn_weights_block1 (tf.Tensor): Attention weights from masked self-attention.
                attn_weights_block2 (tf.Tensor): Attention weights from encoder-decoder attention.
        """
        # BLOCK 1: Masked Self-Attention
        attn_output1, attn_weights_block1 = self.mha1(
            query=x, value=x, key=x, attention_mask=look_ahead_mask, return_attention_scores=True
        )
        out1 = self.layernorm1(x + attn_output1)

        # BLOCK 2: Encoder-Decoder Attention
        attn_output2, attn_weights_block2 = self.mha2(
            query=out1, value=enc_output, key=enc_output, attention_mask=padding_mask, return_attention_scores=True
        )
        out2 = self.layernorm2(out1 + attn_output2)

        # BLOCK 3: Feed-Forward Network
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout_ffn(ffn_output, training=training)
        out3 = self.layernorm3(out2 + ffn_output)

        return out3, attn_weights_block1, attn_weights_block2


In [14]:
class TestDecoderLayer(unittest.TestCase):
    def test_decoder_layer_shapes(self):
        batch_size = 2
        target_seq_len = 5
        input_seq_len = 6
        embedding_dim = 8
        num_heads = 2
        fully_connected_dim = 16

        x = tf.random.uniform((batch_size, target_seq_len, embedding_dim))
        enc_output = tf.random.uniform((batch_size, input_seq_len, embedding_dim))

        layer = DecoderLayer(embedding_dim, num_heads, fully_connected_dim)
        out, attn1, attn2 = layer(x, enc_output, training=False)

        # Check output shape
        self.assertEqual(out.shape, (batch_size, target_seq_len, embedding_dim))

        # Check attention weights shapes
        self.assertEqual(attn1.shape, (batch_size, num_heads, target_seq_len, target_seq_len))
        self.assertEqual(attn2.shape, (batch_size, num_heads, target_seq_len, input_seq_len))

# Run the tests directly in Colab
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..............
----------------------------------------------------------------------
Ran 14 tests in 1.024s

OK


In [15]:
class Decoder(tf.keras.layers.Layer):
    """
    Transformer Decoder consisting of:
    1. Token embedding + positional encoding
    2. Stack of DecoderLayers
    3. Dropout applied to embeddings
    4. Collects attention weights from each decoder layer
    """
    def __init__(self,
                 num_layers: int,
                 embedding_dim: int,
                 num_heads: int,
                 fully_connected_dim: int,
                 target_vocab_size: int,
                 maximum_position_encoding: int,
                 dropout_rate: float = 0.1,
                 layernorm_eps: float = 1e-6):
        """
        Initializes the Decoder.

        Arguments:
            num_layers (int): Number of decoder layers to stack.
            embedding_dim (int): Dimensionality of embeddings.
            num_heads (int): Number of attention heads.
            fully_connected_dim (int): Hidden size of feed-forward network.
            target_vocab_size (int): Vocabulary size of target language.
            maximum_position_encoding (int): Maximum number of positions for positional encoding.
            dropout_rate (float, optional): Dropout rate. Defaults to 0.1.
            layernorm_eps (float, optional): Epsilon for layer normalization. Defaults to 1e-6.
        """
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        self.embedding = Embedding(target_vocab_size, self.embedding_dim)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.embedding_dim)

        self.dec_layers = [
            DecoderLayer(
                embedding_dim=self.embedding_dim,
                num_heads=num_heads,
                fully_connected_dim=fully_connected_dim,
                dropout_rate=dropout_rate,
                layernorm_eps=layernorm_eps
            )
            for _ in range(self.num_layers)
        ]
        self.dropout = Dropout(dropout_rate)

    def call(self,
         x: tf.Tensor,
         enc_output: tf.Tensor,
         training: bool,
         look_ahead_mask: Optional[tf.Tensor] = None,
         padding_mask: Optional[tf.Tensor] = None
        ) -> Tuple[tf.Tensor, Dict[str, tf.Tensor]]:
      """
      Forward pass for the Transformer Decoder.

      This method performs the following steps:
      1. Embeds the input tokens and scales the embeddings.
      2. Adds positional encodings to the embeddings.
      3. Applies dropout to the embeddings.
      4. Passes the embeddings through a stack of DecoderLayers.
      5. Collects attention weights from each decoder layer for analysis.

      Arguments:
          x (tf.Tensor): Input target tensor of shape (batch_size, target_seq_len),
                        containing token IDs.
          enc_output (tf.Tensor): Output from the encoder of shape
                                  (batch_size, input_seq_len, embedding_dim).
          training (bool): Flag indicating whether in training mode (affects dropout).
          look_ahead_mask (Optional[tf.Tensor]): Mask to prevent attention to future positions.
          padding_mask (Optional[tf.Tensor]): Mask to ignore padding positions in encoder-decoder attention.

      Returns:
          Tuple containing:
              x (tf.Tensor): Output of the decoder, shape (batch_size, target_seq_len, embedding_dim).
              attention_weights (Dict[str, tf.Tensor]): Dictionary containing attention weights
                  from each decoder layer:
                  - 'decoder_layer{i}_block1_self_att': Self-attention weights.
                  - 'decoder_layer{i}_block2_decenc_att': Encoder-decoder attention weights.
      """
      seq_len = tf.shape(x)[1]
      attention_weights = {}

      # Token embedding + scale
      x = self.embedding(x)
      x *= tf.math.sqrt(tf.cast(self.embedding_dim, tf.float32))

      # Add positional encoding
      x += self.pos_encoding[:, :seq_len, :]

      # Apply dropout
      x = self.dropout(x, training=training)

      # Pass through decoder layers
      for i, dec_layer in enumerate(self.dec_layers):
          x, block1, block2 = dec_layer(
              x,
              enc_output,
              training=training,
              look_ahead_mask=look_ahead_mask,
              padding_mask=padding_mask
          )
          attention_weights[f'decoder_layer{i+1}_block1_self_att'] = block1
          attention_weights[f'decoder_layer{i+1}_block2_decenc_att'] = block2

      return x, attention_weights


In [16]:
class TestDecoder(unittest.TestCase):
    def test_decoder_shapes(self):
        batch_size = 2
        target_seq_len = 5
        input_seq_len = 6
        embedding_dim = 8
        num_heads = 2
        fully_connected_dim = 16
        num_layers = 2
        vocab_size = 50
        max_pos_encoding = 10

        x = tf.random.uniform((batch_size, target_seq_len), maxval=vocab_size, dtype=tf.int32)
        enc_output = tf.random.uniform((batch_size, input_seq_len, embedding_dim))

        decoder = Decoder(num_layers, embedding_dim, num_heads, fully_connected_dim,
                          vocab_size, max_pos_encoding)
        dec_output, attn_weights = decoder(x, enc_output, training=False)

        self.assertEqual(dec_output.shape, (batch_size, target_seq_len, embedding_dim))
        self.assertEqual(len(attn_weights), num_layers * 2)
        for key, value in attn_weights.items():
            self.assertEqual(value.shape[0], batch_size)  # batch_size matches
            self.assertEqual(value.shape[2], target_seq_len)  # seq_len_q matches

# Run the test in Colab
unittest.main(argv=['first-arg-is-ignored'], exit=False)


...............
----------------------------------------------------------------------
Ran 15 tests in 1.772s

OK


In [17]:
class Transformer(tf.keras.Model):
    """
    Complete Transformer model combining an Encoder and a Decoder.
    This model can be used for sequence-to-sequence tasks such as machine translation.
    """
    def __init__(self,
                 num_layers: int,
                 embedding_dim: int,
                 num_heads: int,
                 fully_connected_dim: int,
                 input_vocab_size: int,
                 target_vocab_size: int,
                 max_positional_encoding_input: int,
                 max_positional_encoding_target: int,
                 dropout_rate: float = 0.1,
                 layernorm_eps: float = 1e-6):
        """
        Initializes the Transformer model with encoder, decoder, and final linear layer.

        Args:
            num_layers (int): Number of encoder and decoder layers.
            embedding_dim (int): Dimensionality of embeddings.
            num_heads (int): Number of attention heads.
            fully_connected_dim (int): Hidden size of feed-forward networks.
            input_vocab_size (int): Vocabulary size for input.
            target_vocab_size (int): Vocabulary size for output.
            max_positional_encoding_input (int): Maximum positional encoding for input.
            max_positional_encoding_target (int): Maximum positional encoding for target.
            dropout_rate (float, optional): Dropout rate. Defaults to 0.1.
            layernorm_eps (float, optional): Epsilon for LayerNormalization. Defaults to 1e-6.
        """
        super().__init__()

        self.encoder = Encoder(
            num_layers=num_layers,
            embedding_dim=embedding_dim,
            num_heads=num_heads,
            fully_connected_dim=fully_connected_dim,
            input_vocab_size=input_vocab_size,
            maximum_position_encoding=max_positional_encoding_input,
            dropout_rate=dropout_rate,
            layernorm_eps=layernorm_eps
        )

        self.decoder = Decoder(
            num_layers=num_layers,
            embedding_dim=embedding_dim,
            num_heads=num_heads,
            fully_connected_dim=fully_connected_dim,
            target_vocab_size=target_vocab_size,
            maximum_position_encoding=max_positional_encoding_target,
            dropout_rate=dropout_rate,
            layernorm_eps=layernorm_eps
        )

        self.final_layer = Dense(target_vocab_size, activation='softmax')

    def call(self,
             input_sentence: tf.Tensor,
             output_sentence: tf.Tensor,
             training: bool,
             enc_padding_mask: Optional[tf.Tensor] = None,
             look_ahead_mask: Optional[tf.Tensor] = None,
             dec_padding_mask: Optional[tf.Tensor] = None
            ) -> Tuple[tf.Tensor, Dict[str, tf.Tensor]]:
        """
        Forward pass of the Transformer.

        Args:
            input_sentence (tf.Tensor): Input tensor of shape (batch_size, input_seq_len) with token indices.
            output_sentence (tf.Tensor): Target tensor of shape (batch_size, target_seq_len) with token indices.
            training (bool): Whether the model is in training mode (affects dropout).
            enc_padding_mask (Optional[tf.Tensor]): Mask for encoder input padding tokens.
            look_ahead_mask (Optional[tf.Tensor]): Mask to prevent attention to future positions.
            dec_padding_mask (Optional[tf.Tensor]): Mask for decoder attention to encoder output padding.

        Returns:
            Tuple[tf.Tensor, Dict[str, tf.Tensor]]:
                - final_output: Tensor of shape (batch_size, target_seq_len, target_vocab_size)
                - attention_weights: Dictionary of attention weights from each decoder layer.
        """
        # Encoder
        enc_output = self.encoder(
            x=input_sentence,
            training=training,
            mask=enc_padding_mask
        )

        # Decoder
        dec_output, attention_weights = self.decoder(
            x=output_sentence,
            enc_output=enc_output,
            training=training,
            look_ahead_mask=look_ahead_mask,
            padding_mask=dec_padding_mask
        )

        # Final linear + softmax
        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


In [18]:
class TestTransformer(unittest.TestCase):
    def test_transformer_shapes(self):
        batch_size = 2
        input_seq_len = 6
        target_seq_len = 5
        embedding_dim = 8
        num_heads = 2
        fully_connected_dim = 16
        num_layers = 2
        input_vocab_size = 50
        target_vocab_size = 60
        max_pos_encoding_input = 10
        max_pos_encoding_target = 10

        input_sentence = tf.random.uniform((batch_size, input_seq_len), maxval=input_vocab_size, dtype=tf.int32)
        output_sentence = tf.random.uniform((batch_size, target_seq_len), maxval=target_vocab_size, dtype=tf.int32)

        transformer = Transformer(num_layers, embedding_dim, num_heads, fully_connected_dim,
                                  input_vocab_size, target_vocab_size,
                                  max_pos_encoding_input, max_pos_encoding_target)

        final_output, attn_weights = transformer(input_sentence, output_sentence, training=False)

        self.assertEqual(final_output.shape, (batch_size, target_seq_len, target_vocab_size))
        self.assertEqual(len(attn_weights), num_layers * 2)  # 2 attention blocks per decoder layer
        for key, value in attn_weights.items():
            self.assertEqual(value.shape[0], batch_size)

# Run the test
unittest.main(argv=['first-arg-is-ignored'], exit=False)

................
----------------------------------------------------------------------
Ran 16 tests in 4.884s

OK
